# 여행 추천 AI

## transformers 이용한 챗봇

In [ ]:
!pip install sentence_transformers
!pip install tfds-korean
!pip install transformers
!pip3 install firebase_admin

In [2]:
import numpy as np
import pandas as pd
from numpy import dot
from numpy.linalg import norm
import urllib.request
from sentence_transformers import SentenceTransformer

In [3]:
# 데이터 셋을 불러옵니다.
train_data = pd.read_excel('chat.xlsx')
# train_data.drop(['Unnamed: 0'], axis=1, inplace=True)
print(len(train_data))
train_data.head()

11876


,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박 4일 놀러 가고 싶다,여행은 언제나 좋죠.,0
3,3박 4일 정도 놀러 가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [4]:
# 문장 임베딩을 얻기 위해서 사전 훈련된 BERT를 로드합니다. 여기서는 한국어도 포함되어 학습된 다국어 모델을 로드합니다.
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

Downloading:   0%|          | 0.00/574 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/731 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/527 [00:00<?, ?B/s]

In [5]:
# 데이터에서 모든 질문열. 즉, train_data['Q']에 대해서 문장 임베딩 값을 구한 후 embedding이라는 새로운 열에 저장합니다.
train_data['embedding'] = train_data.apply(lambda row: model.encode(row.Q), axis = 1)
train_data

,Q,A,label,embedding
0,12시 땡!,하루가 또 가네요.,0,"[0.20179531, -0.034437913, 1.5395736, 0.010697..."
1,1지망 학교 떨어졌어,위로해 드립니다.,0,"[0.077166006, -0.0342782, 0.8624418, 0.0263605..."
2,3박 4일 놀러 가고 싶다,여행은 언제나 좋죠.,0,"[0.10942041, -0.015781153, 1.0562186, 0.028409..."
3,3박 4일 정도 놀러 가고 싶다,여행은 언제나 좋죠.,0,"[0.0943996, -0.048788007, 0.8344291, 0.0192336..."
4,PPL 심하네,눈살이 찌푸려지죠.,0,"[-0.07002864, 0.03196112, 1.491543, 4.3385797e..."
...,...,...,...,...
11871,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2,"[0.15363254, -0.32605517, 0.6987688, 0.1091404..."
11872,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티 나나 봐요.,2,"[0.15363254, -0.32605517, 0.6987688, 0.1091404..."
11873,흑기사 해주는 짝 날.,설렜겠어요.,2,"[-0.007383849, 0.046179295, 0.9020018, -0.0382..."
11874,힘든 연애 좋은 연애라는 게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2,"[-0.074571595, -0.17795657, 0.05517435, 0.4196..."


In [ ]:
# train_test = pd.read_excel('chat.xlsx')
# x_save_load = np.load('embedding.npy', allow_pickle=True)
# train_test['embedding2'] = x_save_load
# train_test

,Q,A,label,embedding2
0,12시 땡!,하루가 또 가네요.,0,"[0.20179531, -0.034437913, 1.5395736, 0.010697..."
1,1지망 학교 떨어졌어,위로해 드립니다.,0,"[0.077166006, -0.0342782, 0.8624418, 0.0263605..."
2,3박 4일 놀러 가고 싶다,여행은 언제나 좋죠.,0,"[0.10942041, -0.015781153, 1.0562186, 0.028409..."
3,3박 4일 정도 놀러 가고 싶다,여행은 언제나 좋죠.,0,"[0.0943996, -0.048788007, 0.8344291, 0.0192336..."
4,PPL 심하네,눈살이 찌푸려지죠.,0,"[-0.07002864, 0.03196112, 1.491543, 4.3385797e..."
...,...,...,...,...
11871,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2,"[0.15363254, -0.32605517, 0.6987688, 0.1091404..."
11872,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티 나나 봐요.,2,"[0.15363254, -0.32605517, 0.6987688, 0.1091404..."
11873,흑기사 해주는 짝 날.,설렜겠어요.,2,"[-0.007383849, 0.046179295, 0.9020018, -0.0382..."
11874,힘든 연애 좋은 연애라는 게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2,"[-0.074571595, -0.17795657, 0.05517435, 0.4196..."


In [6]:
# 두 개의 벡터로부터 코사인 유사도를 구하는 함수 cos_sim를 정의합니다.
def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

In [14]:
# 질문 샘플들의 문장 임베딩 값들을 전부 비교하여 코사인 유사도 값이 가장 높은 질문 샘플을 찾아냅니다.
# 그리고 해당 질문 샘플과 짝이 되는 답변 샘플을 리턴합니다.
def return_answer(question):
    time_start = time.time()
    embedding = model.encode(question)
    train_data['score'] = train_data.apply(lambda x: cos_sim(x['embedding'], embedding), axis=1)
    label = train_data.loc[train_data['score'].idxmax()]['label']
    result_text = train_data.loc[train_data['score'].idxmax()]['A']
    score = train_data.loc[train_data['score'].idxmax()]['score']
    score = int(score * 100)
    time_end = time.time()
    # print(time_end - time_start)

    return score, result_text, label

In [8]:
import time
time_start = time.time()
score, result_text, label = return_answer('안녕?')
time_end = time.time()
# print(time_end - time_start)
print(result_text)

0.2771949768066406
안녕하세요.


In [9]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db
 
 #Firebase database 인증 및 앱 초기화
cred = credentials.Certificate('chat-db.json')
firebase_admin.initialize_app(cred,{
    'databaseURL' : 'https://chat-db-32cde-default-rtdb.firebaseio.com/'
})

In [10]:
# dir2 = db.reference('state/log')
# print(dir2.get()) #json형태로 받아와 진다.

In [11]:
# dir = db.reference('log')
# dir.update({'log':'None'})
# dir.update({'question':result_text})

### 반복 대화 진행

In [15]:
while 1:
    #계속 데이터 읽기
    ref = db.reference('log/question')
    ref2 = db.reference('state/log')
    dir = db.reference('log')
    dir2 = db.reference('state')

    #ref.get()는 #json형태로 받아옴
    if ref.get() == 'stop':
        break
    # 무한 반복 중에서도 원할 때만 값 변경 하기
    elif ref2.get() != "None":
        score, result_text, label = return_answer(ref.get())

        if score >= 30:
            print(result_text)
            print("score : ", score)
            dir.update({'question':result_text})
            dir2.update({'log':'None'})
        else:
            print("잘 못 알아들었어요.")
            dir2.update({'log':'None'})

새로운 도전은 용기가 필요하죠.
score :  98
말동무 챗봇입니다.
score :  99
노는게 제일 좋아요.
score :  96
그러게요. 어느덧 한 달이에요.
score :  98
2주 동안 고생했어요.
score :  95
2주 동안 고생했어요.
score :  97
뭘 먹든 맛있게 드세요!
score :  98
기운을 내세요.
score :  99
재밌겠어요.
score :  99
잼 있는 거는 딸기잼에게 물어봐 주세요.
score :  98
잼 있는 거는 딸기잼에게 물어봐 주세요.
score :  97
저도 딸기 좋아해요. 같이 가요.
score :  95
저도요!
score :  100
저는 좋아요.
score :  100
나 좋다는 사람 만나세요.
score :  97
이별은 사람을 변덕쟁이로 만들기도 하죠.
score :  99


KeyboardInterrupt: ignored